# Notes
WSZYSTKIE PLIKI SĄ IDENTYCZNE. WSZYSTKIE TO EXCEL - 14 kart
Czyszczenie:
•	Wszystkie NaN na 0
•	Skalowanie przed modelem
•	Inżynieria cech (wskaźniki ekonomiczne, różnica miedzy każym z elementów)

TESTY DO PRZEPROWADZENIA:
•	Uwzglednienie wszystkiego
•	Skupienie sie na zmianach kapitałowych
•	Ograniczenie tylko główne wartości
•	Grupowanie po przedsiębiorstwie? 

Podsumowanie co wyciągnąć z pliku:
Dodatkowa kolumna jako nazwa pliku
C – nazwa kolumny w DF


In [50]:
import pandas as pd
import re
from data_utils import *

In [51]:
names_data = pd.read_csv('dataframes/names_gpw.csv')

In [52]:
def clean_column_names(columns):
    cleaned_columns = []
    for column in columns:
        column = column.lower()
        column = re.sub(r'\(.*?\)', '', column)
        column = re.sub(r'[^\w\s]', '', column)
        column = re.sub(r'\s+', '_', column)
        column = column.rstrip('_')
        cleaned_columns.append(column)
    return cleaned_columns

In [53]:
def count_rows_by_filename(df):
    counts = df.groupby('filename').size().reset_index(name='count')
    return counts

# FULL DF CLEARNING

In [54]:
full_data = pd.read_csv("dataframes/dataframe_gpw.csv")

In [55]:
data_gpw = pd.merge(full_data, names_data, on='filename')

In [56]:
data_gpw = data_gpw.sort_values(by=['filename', 'End of period'])

In [57]:
data_gpw.columns = clean_column_names(data_gpw.columns)

In [58]:
count_rows_by_filename(data_gpw)

,filename,count
0,11BIT.xlsx,49
1,3LPSA.xlsx,6
2,3RGAMES.xlsx,48
3,AB.xlsx,70
4,AC.xlsx,51
...,...,...
332,ZPUE.xlsx,95
333,ZREMBCH.xlsx,65
334,ZUE.xlsx,53
335,ZUK.xlsx,67


In [59]:
# Delete all companies under 4 because its not enough to cover any change in assets or liabilities
filenames_to_remove = [
    'SADOVAYA.xlsx',
    'BMW.xlsx',
    'KERNEL.xlsx',
    'SILVANO.xlsx',
    'IMCSA.xlsx',
    'STSHOLDING.xlsx'
]

df_filtered = data_gpw[~data_gpw['filename'].isin(filenames_to_remove)]

## Missing data

In [60]:
data_gpw.isnull().sum()

end_of_period                                                      0
revenues_from_sales                                              945
operating_profitloss                                             737
net_profitloss_attributable_to_equity_holders_of_the_parent      577
total_comprehensive_income_attributable_to_equity_holders      17723
                                                               ...  
noncurrent_financial_liabilities                               19906
financial_liabilities                                          19906
company_name                                                       0
sector                                                            13
ticker                                                             0
Length: 109, dtype: int64

In [61]:
# Sektor NaN
rows_with_nan_in_sektor = data_gpw[data_gpw['sector'].isna()]
rows_with_nan_in_sektor

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,aktywa,...,rate_debt_security,leverage,asset_utilization,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,financial_liabilities,company_name,sector,ticker
5461,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.301800e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
5462,2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.053760e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
5463,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.613490e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
5464,2020-12-31,34515.00000,3929.00000,138.00000,138.0,4114.00000,8513.00000,-3261.00000,-5264.0000,1.640630e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
5465,2021-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.547040e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
5466,2021-12-31,39704.00000,3898.00000,2169.00000,2169.0,4459.00000,5831.00000,-2718.00000,-3195.0000,1.571200e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3LP SA,NaN,3LP_IPO
3071,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.166580e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW AG,NaN,BMW
3072,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.295270e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BMW AG,NaN,BMW
5869,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.065330e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uf Games SA,NaN,UFG_IPO
5870,2020-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.348397e+03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Uf Games SA,NaN,UFG_IPO


# Notes
Manually checking for each company

3LP SA - handel
BMW AG - motoryzacja
Uf Games SA - gry


In [62]:
company_to_sector = {
    '3LP SA': 'handel',
    'BMW AG': 'motoryzacja',
    'Uf Games SA': 'gry'
}

data_gpw['sector'] = data_gpw.apply(lambda row: company_to_sector[row['company_name']] if pd.isna(row['sector']) else row['sector'], axis=1)

In [63]:
# Rest of financial data - fill NaN with 0 (=no change)
data_gpw = data_gpw.fillna(0)

In [64]:
data_gpw.columns.tolist()

['end_of_period',
 'revenues_from_sales',
 'operating_profitloss',
 'net_profitloss_attributable_to_equity_holders_of_the_parent',
 'total_comprehensive_income_attributable_to_equity_holders',
 'depreciation',
 'cash_flow_from_operating_activities',
 'cash_flow_from_investing_activities',
 'cash_flow_from_financing_activities',
 'aktywa',
 'noncurrent_assets',
 'current_assets',
 'equity_shareholders_of_the_parent',
 'noncurrent_liabilities',
 'current_liabilities',
 'number_of_shares',
 'earnings_per_share',
 'total_comprehensive_income_per_share',
 'book_value_per_share',
 'accounting_standard',
 'filename',
 'property_plant_and_equipment',
 'exploration_for_and_evaluation_of_mineral_resources',
 'intangible_assets',
 'goodwill',
 'investment_property',
 'rightofuse_assets',
 'investment_in_affiliates',
 'noncurrent_financial_assets',
 'noncurrent_loans_and_receivables',
 'deferred_income_tax',
 'noncurrent_deferred_charges_and_accruals',
 'noncurrent_derivative_instruments',
 'other

# Change each indicator to % value
return_on_equity
return_on_assets
leverage
asset_utilization
debt_ratio
debt_service_ratio
rate_debt_security
working_capital_ratio

In [65]:
columns_to_convert = [
    'return_on_equity',
    'return_on_assets',
    'leverage',
    'asset_utilization',
    'debt_ratio',
    'debt_service_ratio',
    'rate_debt_security',
    'working_capital_ratio'
]

data_gpw[columns_to_convert] = data_gpw[columns_to_convert] / 100

In [66]:
column_mapping = {
    'financial_liabilities': 'total_liability',
    'aktywa': 'total_assets'
}

data_gpw = data_gpw.rename(columns=column_mapping)

In [67]:
data_gpw['company_name'].unique()

array(['11 bit studios SA', '3LP SA', '3R Games SA', 'AB SA', 'AC SA',
       'Action SA', 'Adiuvo Investments SA', 'Agora SA', 'Agroton plc.',
       'AUGA Group AB', 'All in! Games SA', 'Ailleron SA',
       'Airway Medix SA', 'Alumetal SA', 'Ambra SA', 'Amica SA',
       'Ampli SA w upadłości układowej', 'AmRest Holdings SE',
       'Answear.com SA', 'Apator SA', 'Aplisens SA', 'APS Energia SA',
       'Arctic Paper SA', 'Arteria SA', 'Artifex Mundi SA',
       'ASBISc Enterprises plc.', 'ASM Group SA', 'Asseco BS SA',
       'Asseco Poland SA', 'Asseco SEE SA', 'Atende SA',
       'Atlanta Poland SA', 'ATM Grupa SA', 'Atrem SA', 'Auto Partner SA',
       'Grupa Azoty SA', 'British Automotive Holding SA',
       'Elektrociepłownia Będzin SA', 'Benefit Systems SA', 'Berling SA',
       'Betacom SA', 'Big Cheese Studio SA', 'Bioceltix SA',
       'BioMaxima SA', 'Biomed Lublin SA', 'Bio Planet SA', 'Bioton SA',
       'BMW AG', 'LW Bogdanka SA', 'BoomBit SA', 'Boryszew SA',
       'Bo

## Feature Engineering

In [68]:
data_gpw

,end_of_period,revenues_from_sales,operating_profitloss,net_profitloss_attributable_to_equity_holders_of_the_parent,total_comprehensive_income_attributable_to_equity_holders,depreciation,cash_flow_from_operating_activities,cash_flow_from_investing_activities,cash_flow_from_financing_activities,total_assets,...,rate_debt_security,leverage,asset_utilization,load_gross_profit,current_financial_liabilities,noncurrent_financial_liabilities,total_liability,company_name,sector,ticker
6855,2010-03-31,66.0,0.0,-16.0,0.0,29.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
6856,2010-09-30,129.0,0.0,-26.0,0.0,6.0,0.0,0.0,0.0,1411.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
6857,2010-12-31,228.0,0.0,-198.0,0.0,0.0,0.0,0.0,0.0,1352.36,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
6858,2011-03-31,144.0,0.0,-50.0,0.0,0.0,0.0,0.0,0.0,1342.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
6859,2011-06-30,959.0,0.0,377.0,0.0,7.0,0.0,0.0,0.0,1659.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11 bit studios SA,gry,11B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7374,2021-09-30,995385.0,103668.0,104744.0,106250.0,36985.0,354390.0,-22037.0,-433167.0,2649304.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC
7375,2021-12-31,733974.0,46890.0,32175.0,42434.0,42311.0,50929.0,-65863.0,30091.0,2508447.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC
7376,2022-03-31,696778.0,6649.0,-1246.0,41367.0,38391.0,13967.0,-33280.0,57099.0,2741947.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC
7377,2022-06-30,1209418.0,101579.0,92625.0,45386.0,39932.0,101877.0,-21179.0,-81990.0,2981239.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Grupa Żywiec SA,napoje,ZWC


In [69]:
#TODO do kolumny ticker dodać string '.WA'

file_path = 'dataframes/names_gpw.csv'
data_gpw = pd.read_csv(file_path)

if 'Ticker' in data_gpw.columns:
     data_gpw['Ticker'] = data_gpw['Ticker'].astype(str) + ".WA"
     data_gpw.to_csv('dataframes/changed_names_gpw.csv', index=False)
     result = "The file has been written with '.WA' appended to the 'ticker' column."
else:
     result = "Column 'ticker' does not exist in the DataFrame."

result

"The file has been written with '.WA' appended to the 'ticker' column."

In [70]:
#TODO wywołać funkcję get_market_value i dodać tą wartość do nowej kolumny 'target'

In [71]:
#TODO sortujesz wartość w df'ie po nazwie firmy i dacie
# wygodnie będzie przełożyć nazwe firmy i end_of_period na przednie kolumny
data_gpw = data_gpw.sort_values(by=['filename', 'end_of_period'])

KeyError: 'end_of_period'

In [ ]:
#TODO FEATURE ENGINEERING trzeba dodać kolumne, która liczy wartości z poprzedniego kwartału, roku, 2 lat
# UWAGA: 

# Change in capital or assets structure
# feature: change in last quarter for each company (every major data attribute)
# feature: change in last two quarters for each company (every major data attribute)
# feature: change in last four quarters for each company (every major data attribute)

# NN DATA CLEANING (to póki co nieistotne - trzeba skończyć pelny plik, a potem wybrać sobie odpowiednie kolumny)

In [ ]:
#TODO tu będzie mniejszy dataframe z bardzie dokładnymi danymi do testowania wyników (pewnie da lepsze bo unikamy dużo szumu ze zbędnych kolumn)
#TODO ten df ma być zrobiony z kolumn poniżej

#column_mapping = {
#    'financial_liabilities': 'total_liability',
#    'aktywa': 'total_assets'
#}

In [ ]:
columns_to_keep = [
    'filename',
    # Additional Information
    'Current financial liabilities',
    'Non-current financial liabilities',
    'Financial liabilities (total)',
    
    # DuPont Indicators
    'Return on equity (ROE)',
    'Return on assets (ROA)',
    'Leverage (EM)',
    'Net profit margin',
    'Asset utilization (AU)', # assets_utilization

    # Indicators
    'Current ratio', #curent_ratio
    'Quick ratio',
    'Cash ratio',
    'Receivables turnover',
    'Inventory turnover',
    'The operating cycle',
    'Rotation commitments',
    'Cash conversion cycle',
    'Rotation assets',
    'Rotation of assets',
    'Assets ratio',
    'Debt ratio',
    'Debt service ratio',
    'Rate debt security',
    
    # Basic Information
    'End of period',
    
    # Aktywa (Assets)
    'Non-current assets',
    'Current assets',
    
    # Pasywa (Liabilities)
    'Equity shareholders of the parent',
    'Non-current liabilities',
    'Current liabilities'
]

In [ ]:
nn_data = pd.read_csv("dataframes/nn_gpw_data.csv")

In [ ]:
nn_merged_df = pd.merge(nn_data, names_data, on='filename')

In [ ]:
nn_merged_df = nn_merged_df.sort_values(by=['filename', 'End of period'])

In [ ]:
nn_merged_df.columns = clean_column_names(nn_merged_df.columns)

# MINI DATAFRAME

In [ ]:
#TODO tu będzie jeszcze mniej informacji, tylko te absolutnie najważniejsze (liczę że da najlepsze wyniki)

In [ ]:
#TODO jeszcze muszę ogarnąć jakie kolumny tu będą